<h1>Neighborhoods in Toronto</h1>
<p>I am going to explore and cluster the neighborhoods in Toronto based on wikipedia and foursquere data.</p>

<h3>Create dataframe</h3>

Firstly, I downloaded the table and created csv file. Let's read data into pandas dataframe.

In [1]:
import pandas as pd
codes=pd.read_csv("postal_codes.csv",sep=";")
codes.head()

,Postal Code,Borough,Neighborhood
0,M6C,York,Humewood-Cedarvale
1,M6E,York,Caledonia-Fairbanks
2,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn"
3,M6N,York,"Runnymede, The Junction North"
4,M9N,York,Weston


<h3>Pre-processing</h3>

Removing cells with a borough that is Not assigned.

In [2]:
codes.shape

(180, 3)

In [3]:
import numpy as np
codes=codes.drop(np.where(codes["Borough"]=="Not assigned")[0])
codes.shape

(103, 3)

<h3>Adding coordinates</h3>

Installing pgeocode

In [4]:
! pip install pgeocode

In [5]:
import pgeocode
nomi = pgeocode.Nominatim('ca')

#Func for getting latitude
def get_lat(code):
    return nomi.query_postal_code(code).latitude

#Func for getting longitude
def get_lon(code):
    return nomi.query_postal_code(code).longitude

codes["Latitude"]=codes["Postal Code"].apply(get_lat)
codes["Longitude"]=codes["Postal Code"].apply(get_lon)

codes.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M6C,York,Humewood-Cedarvale,43.6915,-79.4307
1,M6E,York,Caledonia-Fairbanks,43.6889,-79.4507
2,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.6934,-79.4857
3,M6N,York,"Runnymede, The Junction North",43.6748,-79.4839
4,M9N,York,Weston,43.7068,-79.5170


Check our data

In [6]:
codes.isna().sum()

Postal Code     0
Borough         0
Neighborhood    0
Latitude        1
Longitude       1
dtype: int64

One code has not been found in pgeocod. Lets delete it

In [7]:
codes.dropna(axis=0,inplace=True)
codes.shape

(102, 5)

<h3>Creating a map of Toronto</h3>

Get Toronto coordinates

In [15]:
from geopy.geocoders import Nominatim

address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [16]:
import folium

# create map using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(codes['Latitude'], codes['Longitude'], codes['Borough'], codes['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h3>Downtown Toronto - preprocessing data</h3>

I decided to explore Downtown Toronto. Firstly, I create the another dataframe for it.

In [14]:
downtown_df = codes[codes['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
4,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754


Get Downtown coordinates

In [17]:
address = 'Toronto, Downtown Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


Creating the map of Toronto Downtown

In [20]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(downtown_df['Latitude'], downtown_df['Longitude'],downtown_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

<h3>Explore Neighborhoods in Toronto Downtown</h3>

I use the Foursquare API to explore the neighborhoods and segment them.

In [29]:
import requests # library to handle requests

CLIENT_ID = '44TCPS2WRSRAMAYRI3K1AEA2SAZ1EIDR3BUQX5RSVSFSYE13' 
CLIENT_SECRET = 'BOOEV2DB1NV2ORS3BQ101Z4QV1NQB0YNAYE1AR1PCT3NEQXJ' 
VERSION = '20180605' 

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

Function for getting venues

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Create the dataframe for Toronto Downtown using my function

In [31]:
downtown_venues = getNearbyVenues(names=downtown_df['Neighborhood'],
                                  latitudes=downtown_df['Latitude'],
                                  longitudes=downtown_df['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


Let's check the size of the resulting dataframe

In [32]:
print(downtown_venues.shape)
downtown_venues.head()

(1164, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.6555,-79.3626,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.6555,-79.3626,Body Blitz Spa East,43.654735,-79.359874,Spa


Let's check how many venues were returned for each neighborhood

In [33]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,89,89,89,89,89,89
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",57,57,57,57,57,57
Central Bay Street,63,63,63,63,63,63
Christie,11,11,11,11,11,11
Church and Wellesley,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",4,4,4,4,4,4


Find out how many unique categories can be curated from all the returned venues

In [34]:
len(downtown_venues['Venue Category'].unique())

182

<h3>Analyze Each Neighborhood<h3>

In [49]:
# one hot encoding
downtn_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtn_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

downtn_onehot.head()

,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Examine the dataframe size.

In [51]:
downtn_onehot.shape

(1164, 182)

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [52]:
downtn_grouped = downtn_onehot.groupby('Neighborhood').mean().reset_index()
downtn_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.011236,0.022472,0.000000,0.000000,0.000000,0.011236,0.000000,0.011236,...,0.00,0.011236,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.011236
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.017544,0.00,0.017544
2,Central Bay Street,0.000000,0.000000,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.015873,0.015873,0.00,0.015873,0.000000,0.000000,0.00,0.000000
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000
4,Church and Wellesley,0.013158,0.013158,0.000000,0.000000,0.013158,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.013158,0.013158,0.00,0.026316
5,"Commerce Court, Victoria Hotel",0.000000,0.030000,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,...,0.00,0.010000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.01,0.000000
6,"First Canadian Place, Underground city",0.000000,0.030000,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,...,0.00,0.010000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.01,0.000000
7,"Garden District, Ryerson",0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.010000,0.010000,0.00,0.010000,0.000000,0.000000,0.00,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.040000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.060000,0.000000,0.040000,0.00,0.020000,0.000000,0.000000,0.00,0.000000


Print each neighborhood along with the top 5 most common venues

In [53]:
num_top_venues = 5

for hood in downtn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtn_grouped[downtn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1          Café  0.06
2         Hotel  0.04
3      Beer Bar  0.03
4  Cocktail Bar  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.07
1                Café  0.05
2  Italian Restaurant  0.05
3                 Bar  0.05
4              Bakery  0.04


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0.19
1                       Café  0.06
2        Japanese Restaurant  0.05
3  Middle Eastern Restaurant  0.03
4             Sandwich Place  0.03


----Christie----
           venue  freq
0           Café  0.27
1  Grocery Store  0.27
2     Baby Store  0.09
3    Candy Store  0.09
4    Coffee Shop  0.09


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.05
1     Sushi Restaurant  0.05
2  Japanese Restaurant  0.05
3           Restaura

A function to sort the venues in descending order.

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood.

In [86]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtn_grouped['Neighborhood']

for ind in np.arange(downtn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Hotel,Beer Bar,Seafood Restaurant,Restaurant,Cocktail Bar,Bakery,Japanese Restaurant,Art Gallery
1,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Bar,Italian Restaurant,Café,French Restaurant,Park,Speakeasy,Bakery,Bank,Restaurant
2,Central Bay Street,Coffee Shop,Café,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Sandwich Place,Italian Restaurant,Comic Shop,Restaurant,Ramen Restaurant
3,Christie,Grocery Store,Café,Playground,Baby Store,Coffee Shop,Candy Store,Park,General Travel,Gift Shop,Gourmet Shop
4,Church and Wellesley,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Hotel,Men's Store,Mediterranean Restaurant,Grocery Store


<h3>Cluster Neighborhoods</h3>

Run k-means to cluster the neighborhood into 3 clusters.

In [88]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 3

downtn_grouped_clustering = downtn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 0, 2, 2, 2, 2, 1, 2], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [89]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtn_merged = downtown_df
downtn_merged = downtn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtn_merged.head() 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,2,Coffee Shop,Breakfast Spot,Restaurant,Yoga Studio,Spa,Bakery,Beer Store,Distribution Center,Electronics Store,Event Space
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,2,Sushi Restaurant,Burrito Place,Italian Restaurant,Bubble Tea Shop,Burger Joint,Café,Ethiopian Restaurant,Martial Arts Dojo,Coffee Shop,Ramen Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,2,Coffee Shop,Clothing Store,Italian Restaurant,Café,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Movie Theater,Bakery,Bubble Tea Shop
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,2,Café,Coffee Shop,Seafood Restaurant,Cocktail Bar,Italian Restaurant,Gastropub,American Restaurant,Restaurant,Clothing Store,Department Store
4,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754,2,Coffee Shop,Café,Hotel,Beer Bar,Seafood Restaurant,Restaurant,Cocktail Bar,Bakery,Japanese Restaurant,Art Gallery


Finally, let's visualize the resulting clusters

In [90]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtn_merged['Latitude'], downtn_merged['Longitude'], downtn_merged['Neighborhood'], downtn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>Examine Clusters</h3>

Cluster 1 - residential areas

In [91]:
downtn_merged.loc[downtn_merged['Cluster Labels'] == 0, downtn_merged.columns[[1] + list(range(3, downtn_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,43.6683,-79.4205,0,Grocery Store,Café,Playground,Baby Store,Coffee Shop,Candy Store,Park,General Travel,Gift Shop,Gourmet Shop
14,Downtown Toronto,43.6827,-79.3730,0,Playground,Grocery Store,Park,Candy Store,Dog Run,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


Cluster 2 - harbor

In [95]:
downtn_merged.loc[downtn_merged['Cluster Labels'] == 1, downtn_merged.columns[[1] + list(range(3, downtn_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Downtown Toronto,43.623,-79.3936,1,Harbor / Marina,Park,Café,Music Venue,Donut Shop,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store


Cluster 3 - business center

In [94]:
downtn_merged.loc[downtn_merged['Cluster Labels'] == 2, downtn_merged.columns[[1] + list(range(3, downtn_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,43.6555,-79.3626,2,Coffee Shop,Breakfast Spot,Restaurant,Yoga Studio,Spa,Bakery,Beer Store,Distribution Center,Electronics Store,Event Space
1,Downtown Toronto,43.6641,-79.3889,2,Sushi Restaurant,Burrito Place,Italian Restaurant,Bubble Tea Shop,Burger Joint,Café,Ethiopian Restaurant,Martial Arts Dojo,Coffee Shop,Ramen Restaurant
2,Downtown Toronto,43.6572,-79.3783,2,Coffee Shop,Clothing Store,Italian Restaurant,Café,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Movie Theater,Bakery,Bubble Tea Shop
3,Downtown Toronto,43.6513,-79.3756,2,Café,Coffee Shop,Seafood Restaurant,Cocktail Bar,Italian Restaurant,Gastropub,American Restaurant,Restaurant,Clothing Store,Department Store
4,Downtown Toronto,43.6456,-79.3754,2,Coffee Shop,Café,Hotel,Beer Bar,Seafood Restaurant,Restaurant,Cocktail Bar,Bakery,Japanese Restaurant,Art Gallery
5,Downtown Toronto,43.6564,-79.3860,2,Coffee Shop,Café,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Sandwich Place,Italian Restaurant,Comic Shop,Restaurant,Ramen Restaurant
7,Downtown Toronto,43.6496,-79.3833,2,Café,Coffee Shop,Restaurant,Gym,American Restaurant,Steakhouse,Asian Restaurant,Salad Place,Japanese Restaurant,Thai Restaurant
9,Downtown Toronto,43.6469,-79.3823,2,Coffee Shop,Hotel,Café,Restaurant,Seafood Restaurant,Deli / Bodega,Salad Place,Japanese Restaurant,American Restaurant,Beer Bar
10,Downtown Toronto,43.6492,-79.3823,2,Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Steakhouse,Japanese Restaurant,Asian Restaurant,Salad Place
11,Downtown Toronto,43.6629,-79.3987,2,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Bakery,Bookstore,Yoga Studio,Beer Bar,Coffee Shop,Dessert Shop
